In [4]:
# !python -m pip install mlagents==0.28.0
# !python -m pip install gym
# !cd "/Users/aditya/Documents/GitHub/game_creation_research/ml-agents/gym-unity" && pip3 install -e .

import mlagents
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from gym_unity.envs import UnityToGymWrapper

In [6]:
SERVER_BUILD = "../Builds/MLAgent_View_21April22_server.app"
GRAPHICAL_BUILD = "../Builds/MLAgent_View_21April22.app"
GYM_BUILD = "../Builds/Gym_View_22April22.app"

channel = EngineConfigurationChannel()
channel.set_configuration_parameters(time_scale = 1.0, quality_level=1, target_frame_rate=5)
env = UnityEnvironment(file_name=GYM_BUILD, seed=1, side_channels=[channel], worker_id=1)

# Start interacting with the environment.
env.reset()
env = UnityToGymWrapper(env, allow_multiple_obs=True)

UnityEnvironmentException: Couldn't launch the ../Builds/Gym_View_22April22.app environment. Provided filename does not match any environments.

In [3]:
env.step(action=[0.1, 0.5, 0, 0, 0, 0])

([array([ 5.34      , -2.65      , -2.8114905 ,  0.10777473,  5.34      ,
         -1.25      ,  5.34      ,  0.24      ,  5.34      ,  3.26      ,
          1.        , -4.1317406 ,  0.        ], dtype=float32)],
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7fc488aad550>})

In [34]:
env.step(action=[0.1, 0.5, 0.1, 0.1, 1, 0])

([array([ 5.34      , -2.65      , -2.8114905 ,  0.10777473,  5.34      ,
         -1.25      ,  5.34      ,  0.24      ,  1.        ,  1.        ,
          1.        , -2.9022753 ,  0.        ], dtype=float32)],
 0.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7fbff22130d0>})

In [3]:
env.close()